<a href="https://colab.research.google.com/github/zeyadhabib/Tf-Speech-Recognition-Model/blob/main/TF_Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q tensorflow-io
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np

     |████████████████████████████████| 22.3MB 36.9MB/s 


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle 

     |████████████████████████████████| 61kB 4.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-cp36-none-any.whl size=73275 sha256=ae18e941730214f9e74bb736b58b5e62581a2776ac6cd48a2eac55130c85081f
  Stored in directory: /root/.cache/pip/wheels/94/a7/09/68dc83c7c14fdbdf5d3f2b2da5b87e587bfc1e85df69b1130c
Successfully built kaggle
  Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8


In [ ]:
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c tensorflow-speech-recognition-challenge

Saving kaggle.json to kaggle.json
100% 3.49G/3.50G [01:05<00:00, 39.3MB/s]
100% 3.50G/3.50G [01:05<00:00, 57.2MB/s]


In [ ]:
! unzip tensorflow-speech-recognition-challenge.zip
# ! mkdir train
! ls

Archive:  tensorflow-speech-recognition-challenge.zip
  inflating: link_to_gcp_credits_form.txt  
  inflating: sample_submission.7z    
  inflating: test.7z                 
  inflating: train.7z                
kaggle.json		      tensorflow-speech-recognition-challenge.zip
link_to_gcp_credits_form.txt  test.7z
sample_data		      train.7z
sample_submission.7z


In [ ]:
!7z x train.7z -o.
#! 7z x test.7z -o. #skip loading test data for now


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1121103842 bytes (1070 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 1121103842
Headers Size = 389133
Method = Delta LZMA2:24
Solid = +
Blocks = 2

  0%      0% 39 - train/audio/_background_noise_/dude_miaowing.wav                                                            0% 41 - train/audio/_background_noise_/pink_noise.wav                                                         0% 42 - train/audio/_background_noise_/running_tap.wav

In [ ]:
! pwd

/content


In [ ]:
!ls

kaggle.json		      tensorflow-speech-recognition-challenge.zip
link_to_gcp_credits_form.txt  test.7z
sample_data		      train
sample_submission.7z	      train.7z


In [ ]:
### is this what you wanted ali?? better than linear search i guess
def load_test_file_set(test_file_path='./train/testing_list.txt'):
    #open the txt file containing the pathes of files that should be added to test dataset
    #convert the list to set datastructure
    test_files = open(test_file_path).read().splitlines()
    return set(test_files)

def load_validation_file_set(val_file_path='./train/validation_list.txt'):
    #open the txt file containing the pathes of files that should be added to validation dataset
    #convert the list to set datastructure
    validation_files = open(val_file_path).read().splitlines()
    return set(validation_files)

def is_test_file(test_set, file_path):
    return file_path in test_set

def is_validation_file(val_set, file_path):
    return file_path in val_set




In [ ]:
def cut_into_one_sec_segment(audio , sampling_rate):
    length = audio.shape[0]
    #pad audio with zeros

    num_segments = int(np.ceil(length/sampling_rate))
    audio = np.pad(audio, ((0, sampling_rate * num_segments-audio.shape[0]), (0, 0)), 'constant', constant_values=(0, ))
    segments = audio[0:  sampling_rate, :]
    for i in range(1,num_segments):
        seg = audio[ i * sampling_rate : ((i + 1) * sampling_rate ) , :]
        segments = np.hstack((segments, seg))
    return segments

In [ ]:
import os
#outputs  an array of shape(16000, some number)
def load_background_noise(sampling_rate = 16000):##Pads zeros to each background wave file
    #read background music then cut then add to dataset
    bg_noise_dirpath = './train/audio/_background_noise_/'
    segments = None
    first_iter = True
    for filename in os.listdir(bg_noise_dirpath):
        if (filename.endswith('.wav')):
            full_file_path = os.path.join(bg_noise_dirpath, filename)
            audio = tfio.audio.AudioIOTensor(full_file_path).to_tensor().numpy()
            
            #first iter return first segment into segments array
            if first_iter == True:
                first_iter = False
                #convert noise audio to 1 second long audio segments with zero padding
                segments = cut_into_one_sec_segment(audio, sampling_rate)
            else:#2nd+ iter stack the new segments
                #convert noise audio to 1 second long audio segments with zero padding
                new_segments = cut_into_one_sec_segment(audio, sampling_rate)
                segments = np.hstack((segments, new_segments))
    
    return segments

background_dataset = load_background_noise()
print(background_dataset.shape)

(16000, 402)


In [ ]:
#CELL TO RUN to show output
## this the cell to USe to load the train data

# ALI ZEYAD RUN THIS


# ! mv train/audio/_background_noise_ .
###classes = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go','unknown']
num_wav_files_with_bgnoise = 64727 #(computed before using a forloop)
#number of wav files not counting the background noise clips that have to broken down into smaller units
#FS is 16000 for all inputs

import os

X_train = []
Y_train = []

X_val   = []
Y_val   = []
X_test  = []
Y_test  = []

def read_train_audio(folder_path='./train/audio'):
    
    global X_train, X_val, Y_train, Y_val

    classes = np.array(['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown'])

    all_labels = np.array(['_background_noise_', 'dog', 'four', 'seven','three',
                       'wow', 'bed', 'marvin', 'sheila', 'tree',
                       'bird', 'eight', 'happy', 'nine', 'one', 'six', 'two', 'zero', 'cat',
                       'five', 'house','yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go'])


    label_id = {}
    for i in range(len(all_labels)):
        label_id[all_labels[i]] = i
        
    class_id = {}
    for i in range(len(classes)):
        class_id[classes[i]] = i
    test_file_set = load_test_file_set()
    validation_file_set = load_validation_file_set()
    
    for label in os.listdir(folder_path): #loop on all folders in the audio folder
        dir_path = os.path.join(folder_path, label)
        if os.path.isfile(dir_path):#skip any files
            continue
        for filename in os.listdir(dir_path): #loop on all files in folder
            if label == '_background_noise_':
                #background noise is loaded in a seperate
                break
            else:
                
                full_file_path = os.path.join(dir_path, filename)
                if os.path.isdir(full_file_path):
                    continue
                audio = tfio.audio.AudioIOTensor(full_file_path).to_tensor().numpy()
                file_rel_path = os.path.join(label, filename)
                if is_validation_file(validation_file_set, file_rel_path):
                    X_val.append(np.pad(audio, ((0, 16000-audio.shape[0]), (0, 0)), 'constant', constant_values=(0, )))                    
                    Y_val.append(label_id.get(label, len(all_labels) - 1))
                elif is_test_file(test_file_set, file_rel_path):
                    #add to test data ??
                    pass
                else:
                    X_train.append(np.pad(audio, ((0, 16000-audio.shape[0]), (0, 0)), 'constant', constant_values=(0, )))                    
                    Y_train.append(label_id.get(label, len(all_labels) - 1))


    
   
    X_train = np.array(X_train)
    X_val = np.array(X_val)
    Y_val = np.array(Y_val)
    Y_val = Y_val.reshape(Y_val.shape[0], 1)


    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
    X_val = X_val.reshape(X_val.shape[0], X_val.shape[1])

    bgnoise_train_data = load_background_noise()
    bgnoise_train_data = bgnoise_train_data.T
    print(bgnoise_train_data.shape)
    y_bgnoise = [label_id['_background_noise_']] * bgnoise_train_data.shape[0]
    Y_train.extend(y_bgnoise)
    Y_train = np.array(Y_train)
    Y_train = Y_train.reshape(Y_train.shape[0], 1)

    X_train = np.vstack((X_train, bgnoise_train_data))
    print(X_train.shape)


read_train_audio()

(402, 16000)
(51490, 16000)


In [ ]:
print('X Train (words + background noise) Dimensions:', X_train.shape)
print('Y Train' , Y_train.shape)

print('X Validation' , X_val.shape)
print('Y Validation' , Y_val.shape)


X Train (words + background noise) Dimensions: (51490, 16000)
Y Train (51490, 1)
X Validation (6798, 16000)
Y Validation (6798, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, Dense
from tensorflow.keras.layers import Bidirectional, LSTM, TimeDistributed


def get_SR_Model(num_classes: int):


  ################################################################
    #best hyper params:
        # conv_1  (128, 31, strides=8) --> Dropout = 0.2
        # conv_2  (256, 31, strides=8) --> Dropout = 0.2
        # lstm_   (512, return_sequences=False) --> Dropout = 0.2
        # dense_1 (512)  --> Dropout = 0.4
  ################################################################


    X_input = Input(shape=(16000, 1))
    X = Conv1D(128, 31, strides=8, activation='relu', name='conv_1')(X_input) 
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Conv1D(256, 31, strides=8, activation='relu', name='conv_2')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = LSTM(units=512, return_sequences=False, name='lstm_')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Dense(512, activation='relu', name='dense_1')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.4)(X)
    X = Dense(num_classes, activation='softmax', name='dense_2')(X)

    return Model(inputs=[X_input], outputs=[X])

model = get_SR_Model(np.max(Y_train) + 1)

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics


model.compile(loss=losses.sparse_categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16000, 1)]        0         
_________________________________________________________________
conv_1 (Conv1D)              (None, 1997, 128)         4096      
_________________________________________________________________
batch_normalization (BatchNo (None, 1997, 128)         512       
_________________________________________________________________
dropout (Dropout)            (None, 1997, 128)         0         
_________________________________________________________________
conv_2 (Conv1D)              (None, 246, 256)          1016064   
_________________________________________________________________
batch_normalization_1 (Batch (None, 246, 256)          1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 246, 256)         

In [ ]:
## best batch_size = 64
history = model.fit(X_train, Y_train, batch_size=64, validation_data=(X_val, Y_val), shuffle=True, epochs=25)

Epoch 1/25
805/805 [==============================] - 88s 109ms/step - loss: 3.6653 - accuracy: 0.0666 - val_loss: 3.3447 - val_accuracy: 0.0947
Epoch 2/25
805/805 [==============================] - 94s 116ms/step - loss: 2.2923 - accuracy: 0.3252 - val_loss: 2.6637 - val_accuracy: 0.2482
Epoch 3/25
805/805 [==============================] - 94s 116ms/step - loss: 1.4018 - accuracy: 0.5711 - val_loss: 1.5007 - val_accuracy: 0.5547
Epoch 4/25
805/805 [==============================] - 93s 116ms/step - loss: 0.9610 - accuracy: 0.7080 - val_loss: 0.9481 - val_accuracy: 0.7224
Epoch 5/25
805/805 [==============================] - 93s 116ms/step - loss: 0.7321 - accuracy: 0.7784 - val_loss: 0.8534 - val_accuracy: 0.7583
Epoch 6/25
805/805 [==============================] - 93s 116ms/step - loss: 0.5984 - accuracy: 0.8201 - val_loss: 0.5702 - val_accuracy: 0.8330
Epoch 7/25
805/805 [==============================] - 94s 116ms/step - loss: 0.5238 - accuracy: 0.8420 - val_loss: 0.5777 - val_ac

In [ ]:
res = np.column_stack((np.argmax(model.predict(X_val), axis=1), Y_val))

In [ ]:
print(res[765:815])

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
import tensorflow as tf
import numpy as np

classes = np.array(['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown'])

all_labels = np.array(['_background_noise_', 'dog', 'four', 'seven','three',
                       'wow', 'bed', 'marvin', 'sheila', 'tree',
                       'bird', 'eight', 'happy', 'nine', 'one', 'six', 'two', 'zero', 'cat',
                       'five', 'house','yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go'])


def from_31_to_11(x):
    mask = np.array([False for i in range(31)])
    inv_mask = np.array([True for i in range(31)])
    for name in classes:
        index = np.where(all_labels == name)[0]
        mask[index] = True
        inv_mask[index] = False
    one_hot_11 = tf.boolean_mask(x, mask)
    unk = tf.math.reduce_sum(tf.boolean_mask(x, inv_mask), axis=0, keepdims=True)
    one_hot_11 = tf.concat([one_hot_11, unk], axis=0)
    return one_hot_11

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Lambda
X_input = Input(shape=(16000, 1))
X = model(X_input)
X = Lambda(from_31_to_11)(X)
new_Model = Model(inputs=[X_input], outputs=[X])

print(new_Model.predict(X_val)[0])

fatal: not a git repository (or any of the parent directories): .git
